In [1]:
from bspump.jupyter import *
import bspump.http.web.source
import bspump.file
from bspump.abc.lookup import Lookup
import time
import json

BitSwan BSPump version devel


In [2]:
words = {
    "bap": "lol",
    "baz": "baf",
}

class WordLookup(Lookup):
    def get(self, key):
        return words.get(key)

register_lookup(WordLookup)

In [3]:
new_pipeline("LookupPipeline")

In [4]:
@register_source
def source(app, pipeline):
   return bspump.http.web.source.WebHookSource(
       app,
       pipeline,
       config = {
           "port": 8080,
           "path": "/webhook/",
           "secret_qparam": "not-secure",
       })

In [5]:
sample_events([
    b"""{"foo":"bap"}""",
    b"""{"foo":"baz"}"""
])

In [6]:
@step
def parse_json(event):
    return json.loads(event)

{'foo': 'bap'}
{'foo': 'baz'}


In [7]:
class LookupGenerator(bspump.Generator):
    def __init__(self, app, pipeline, id=None, config=None):
        super().__init__(app, pipeline, id, config)
        svc = app.get_service("bspump.PumpService")
        self.word_lookup = svc.locate_lookup("WordLookup")
    
    async def generate(self, context, event, depth):
        event["foo"] = self.word_lookup.get(event["foo"])
        await self.Pipeline.inject(context, event, depth)

@register_generator
def create_lookup_generator(app, pipeline):
    return LookupGenerator(app, pipeline)

{'foo': 'lol'}
{'foo': 'baf'}


In [8]:
@register_sink
def init_sink(app, pipeline):
    return bspump.file.FileCSVSink(app, pipeline)

In [9]:
end_pipeline()